In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
# df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
# df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [3]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [4]:
df_bbidding["price_difference"] = df_bbidding["day_ahead_price"] - df_bbidding["imbalance_price"]

In [5]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, Target_MW, price_diff):
    return price_diff * zb + -0.07*(Target_MW-zb)**2

# Negative revenue function (for minimization)
def negative_revenue(zb, Target_MW, price_difference):
    return -revenue(zb, Target_MW, price_difference)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    Target_MW = row['Target_MW']
    price_difference = row['price_difference']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(Target_MW, price_difference), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



In [6]:
df_bbidding.revenue_optimal.mean()

56847.65672592287

In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [8]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [11]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.loc[:,"market_price_lag96h"] = df_bbidding1["market_price"].shift(192)
df_bbidding1.loc[:,"imbalance_price_lag96h"] = df_bbidding1["imbalance_price"].shift(192)
df_bbidding1.loc[:,"day_ahead_price_lag1week"] = df_bbidding1["day_ahead_price"].shift(336)
df_bbidding1.loc[:,"volume_lag96h"] = df_bbidding1["volume"].shift(192)
df_bbidding1["price_difference"] = df_bbidding1["day_ahead_price"] - df_bbidding1["imbalance_price"]
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,price_difference
336,2021-03-04 08:00:00+00:00,68.55,73.77,1201.25,17,-0.500000,0.467359,38.647358,73.495896,103.586077,...,163.586503,192.345607,223.088138,303.638908,89.95,7.54,8.00,32.38,755.80,-16.18
337,2021-03-04 08:30:00+00:00,69.25,73.77,1105.35,18,-0.500000,0.467359,47.682586,78.763138,116.759406,...,175.664818,207.519222,234.407247,320.043294,89.00,-9.71,7.00,32.38,900.35,-15.23
338,2021-03-04 09:00:00+00:00,75.18,70.86,1245.05,19,-0.707107,0.467359,69.149410,92.206057,131.658071,...,190.143025,222.502626,242.349669,303.730406,92.00,55.58,33.65,17.34,497.60,-21.14
339,2021-03-04 09:30:00+00:00,73.19,70.86,1590.00,20,-0.707107,0.467359,88.297115,137.617950,191.285667,...,268.700178,301.155349,349.524224,401.357202,92.00,57.95,62.70,17.34,554.50,-21.14
340,2021-03-04 10:00:00+00:00,74.12,63.72,1415.30,21,-0.866025,0.467359,138.822877,203.623935,275.650748,...,381.055277,422.775939,512.862891,564.739253,98.00,53.11,34.10,14.25,739.95,-34.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,-10.625771,33.503429,61.177328,...,141.773080,173.530786,203.292972,252.058557,66.01,91.88,126.00,50.21,1271.20,18.40
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-3.847346,7.954975,32.917332,...,91.987493,109.441426,127.084166,187.448090,66.01,90.64,90.64,37.82,1643.15,13.95
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-10.646453,3.459110,29.465151,...,83.669257,100.222843,120.521524,176.329598,66.01,95.15,115.99,37.82,1599.25,13.95
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-9.140216,5.808364,25.305676,...,79.561332,96.108589,116.482859,160.393853,66.01,91.30,125.00,31.66,1619.70,7.83


In [12]:
# Original dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["price_difference"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [13]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [14]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1                  # Dropout rate
learning_rate = 0.0001         # Learning rate for optimizer
batch_size = 32                # Batch size
num_epochs = 500               # Maximum number of epochs
patience = 15                  # Patience for early stopping
rel_improvement_threshold = 0.0001  # Relative improvement threshold

In [15]:
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [16]:
import torch.nn.functional as F  # For MAE loss function

best_val_loss = float('inf')  # Initialize the best validation loss
best_model = None
patience_counter = 0  # For early stopping

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension (for some models)
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Reshape input to add sequence length dimension
            X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

            # Forward pass
            y_pred = model(X_batch)  # (batch_size, output_size)

            # Compute the validation loss using MAE
            val_loss = F.l1_loss(y_pred, y_batch)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Early stopping based on validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model = model.state_dict()  # Save the best model's weights
        patience_counter = 0  # Reset patience counter
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
if best_model is not None:
    model.load_state_dict(best_model)
else:
    print("No improvement was observed during training.")

torch.save(model.state_dict(), 'LSTM_day_ahead_price.pth')

# Test the model using the best model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')
import torch.nn.functional as F  # For MAE loss function

best_val_loss = float('inf')  # Initialize the best validation loss
best_model = None
patience_counter = 0  # For early stopping

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension (for some models)
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Reshape input to add sequence length dimension
            X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

            # Forward pass
            y_pred = model(X_batch)  # (batch_size, output_size)

            # Compute the validation loss using MAE
            val_loss = F.l1_loss(y_pred, y_batch)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Early stopping based on validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model = model.state_dict()  # Save the best model's weights
        patience_counter = 0  # Reset patience counter
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
if best_model is not None:
    model.load_state_dict(best_model)
else:
    print("No improvement was observed during training.")

torch.save(model.state_dict(), 'LSTM_price_difference.pth')

# Test the model using the best model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')


Epoch [1/500], Training Loss: 63.6930
Epoch [1/500], Validation Loss: 69.4740
Best model saved with validation loss: 69.4740
Epoch [2/500], Training Loss: 63.6920
Epoch [2/500], Validation Loss: 69.4738
Best model saved with validation loss: 69.4738
Epoch [3/500], Training Loss: 63.6887
Epoch [3/500], Validation Loss: 69.4747
Epoch [4/500], Training Loss: 63.6762
Epoch [4/500], Validation Loss: 69.4789
Epoch [5/500], Training Loss: 63.6372
Epoch [5/500], Validation Loss: 69.4897
Epoch [6/500], Training Loss: 63.5756
Epoch [6/500], Validation Loss: 69.5089
Epoch [7/500], Training Loss: 63.5260
Epoch [7/500], Validation Loss: 69.5281
Epoch [8/500], Training Loss: 63.4915
Epoch [8/500], Validation Loss: 69.5449
Epoch [9/500], Training Loss: 63.4644
Epoch [9/500], Validation Loss: 69.5573
Epoch [10/500], Training Loss: 63.4416
Epoch [10/500], Validation Loss: 69.5689
Epoch [11/500], Training Loss: 63.4217
Epoch [11/500], Validation Loss: 69.5805
Epoch [12/500], Training Loss: 63.4033
Epoch

In [18]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new = df_api_new.groupby("timestamp").last().reset_index()

In [19]:
from datetime import datetime, timedelta
df_api_new = df_api_new.rename(columns={
    "timestamp": "datetime"
})
df_api_new["datetime"] = pd.to_datetime(df_api_new["datetime"])
min_date = df_api_new.datetime.min() - timedelta(minutes=30)
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
df_half_hourly = pd.DataFrame({"datetime": datetimes})
df_half_hourly["datetime"] = pd.to_datetime(df_half_hourly["datetime"])
df_api_new = pd.merge(df_half_hourly, df_api_new, left_on='datetime', right_on='datetime', how='outer')

In [20]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.day/7)

In [21]:
df_api_new_merged1 = df_api_new_merged[["datetime","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(336)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(192)
df_api_new_merged1["price_difference"] = df_api_new_merged1["day_ahead_price"] - df_api_new_merged1["imbalance_price"]
df_api_new_merged1.dropna(inplace=True)

In [22]:
X_test = df_api_new_merged1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
X_test_tensor = X_test_tensor.unsqueeze(1)  # Adds a sequence length dimension

In [23]:
model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["price_difference_predictions"] = final_test_outputs
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,price_difference,price_difference_predictions
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,326.0,1402.0,1357.0,61.00,12.85,-1.4600,7.20,1457.25,17.47,-13.743317
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,923.0,797.0,487.0,50.00,-3.95,-1.4600,7.20,1367.60,28.47,-13.650796
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,1533.0,597.0,991.0,93.50,-2.74,5.0000,3.87,1473.20,-22.14,-13.165211
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,744.0,1497.0,463.0,93.50,3.85,5.0000,3.87,1799.05,-22.14,-13.939903
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,814.0,487.0,1340.0,93.00,0.03,-2.4304,0.45,1689.75,-20.20,-13.967239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2024-10-21 19:30:00+00:00,71.47,79.79,2221.80,42.0,0.258819,1.000000,263.0,343.0,406.0,...,514.0,547.0,590.0,65.00,86.49,71.5200,91.30,2070.40,14.79,-4.245029
1010,2024-10-21 20:00:00+00:00,87.87,77.99,2156.50,43.0,0.500000,1.000000,266.0,345.0,405.0,...,516.0,551.0,592.0,99.00,81.51,68.0000,85.25,1475.55,-21.01,-5.678957
1011,2024-10-21 20:30:00+00:00,86.70,77.99,2183.15,44.0,0.500000,1.000000,257.0,341.0,397.0,...,515.0,550.0,585.0,99.00,82.97,67.7000,85.25,1298.75,-21.01,-5.414227
1012,2024-10-21 21:00:00+00:00,71.16,74.71,1754.15,45.0,0.707107,1.000000,249.0,338.0,389.0,...,514.0,553.0,587.0,0.94,80.73,70.5000,91.58,1364.60,73.77,-4.606070


In [24]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["price_difference"], mode='lines', name='Imbalance Price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["price_difference_predictions"], mode='lines', name='price_difference Predictions'))
fig.update_layout(title='price_differenceand Predictions', xaxis_title='Timestamp', yaxis_title='price_differencee')
fig.show()

In [25]:
mae = np.mean(np.abs(df_api_new_merged1["price_difference"] - df_api_new_merged1["price_difference_predictions"]))
print(f"MAE for imbalance price predictions: {mae:.4f}")

MAE for imbalance price predictions: 30.9392


In [29]:
df_total_solar.timestamp_utc = pd.to_datetime(df_total_solar.timestamp_utc)
df_total_wind.timestamp_utc = pd.to_datetime(df_total_wind.timestamp_utc)

In [32]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,df_total_solar, left_on='datetime', right_on='timestamp_utc', how='inner')
df_api_new_merged2 = pd.merge(df_api_new_merged2,df_total_wind, left_on='datetime', right_on='timestamp_utc', how='inner')
df_api_new_merged2["Total_generation"] = df_api_new_merged2["generation_mw_x"] + df_api_new_merged2["generation_mw_y"]

In [33]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, Target_MW, price_diff):
    return price_diff * zb + -0.07*(Target_MW-zb)**2

# Negative revenue function (for minimization)
def negative_revenue(zb, Target_MW, price_difference):
    return -revenue(zb, Target_MW, price_difference)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    Target_MW = row['1']
    price_difference = row['price_difference_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(Target_MW, price_difference), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_optimal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_generation'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_generation'] - df_api_new_merged2['optimized_trade']))



In [35]:
df_api_new_merged2.revenue_optimal.mean()

12465.984287701414

In [30]:
df_api_new_merged1.dtypes

datetime                        datetime64[ns, UTC]
market_price                                float64
day_ahead_price                             float64
volume                                      float64
settlement_period                           float64
cos_hour                                    float64
cos_day                                     float64
1                                           float64
2                                           float64
3                                           float64
4                                           float64
5                                           float64
6                                           float64
7                                           float64
8                                           float64
9                                           float64
imbalance_price                             float64
market_price_lag96h                         float64
imbalance_price_lag96h                      float64
day_ahead_pr